In [0]:
#SALVANDO EM: 
# /tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab_Prevenção_de_Acidentes_de_Trabalho_p0                (base inicial)
# /tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab_Prevenção_de_Acidentes_de_Trabalho_p0_data=ddmmaaa   (incrementais)

# /tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab_NODO_RAIZ_p20
# /tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab_NODO_RAIZ_p20_data=ddmmaaaa

# /tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab_Acidentes_de_Trabalho_sem_óbitos_p0
# /tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab_Acidentes_de_Trabalho_sem_óbitos_p0_data=ddmmaaaa

# /tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab_Acidentes_de_Trabalho_com_óbito_p0
# /tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab_Acidentes_de_Trabalho_com_óbito_p0_data=ddmmaaaa

# /tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab_Acidentes_de_Trabalho_Judicializados_p0
# /tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab_Acidentes_de_Trabalho_Judicializados_p0_data=ddmmaaaa

In [0]:
import json
import re
import cni_connectors.adls_gen1_connector as connector
import pyspark.sql.functions as f
import crawler.functions as cf
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, LongType, DoubleType

In [0]:
var_adls_uri = connector.adls_gen1_connect(spark, dbutils, scope="adls_gen2", dynamic_overwrite="dynamic")

fornecedor_dev
dev-execution

In [0]:
#Parametros de entrado do ADF
var_file = {'namespace':'API_InGuru',
'file_folder':'inguro_vig_acidente_trab',
'extension':'csv',
'column_delimiter':',',
'encoding':'UTF-8',
'null_value':''}

#Caminho de Dev
var_dls = {
    "folders":{"landing":"/tmp/dev/uld",
    "error":"/tmp/dev/err",
    "staging":"/tmp/dev/stg",
    "log":"/tmp/dev/log",
    "raw":"/tmp/dev/raw",
    "trusted":"/tmp/dev/trs",
    "business":"/tmp/dev/biz",
    "landing":"/tmp/dev/uld"          }}
#Parametros do ADF (Nativo)
var_adf = {
    "adf_factory_name": "cnibigdatafactory",
    "adf_pipeline_name": "org_raw_vig_acidente_trab",
    "adf_pipeline_run_id": "60ee3485-4a56-4ad1-99ae-666666666",
    "adf_trigger_id": "62bee9e9-acbb-49cc-80f2-666666666",
    "adf_trigger_name": "62bee9e9-acbb-49cc-80f2-66666666",
    "adf_trigger_time": "2020-06-08T01:42:41.5507749Z",
    "adf_trigger_type": "PipelineActivity"
}

In [0]:
#Acesso ao json do CMD
lnd = var_dls['folders']['landing']
raw = var_dls['folders']['raw']

In [0]:
var_source = "{lnd}/{namespace}/{file_folder}/".format(lnd=lnd, namespace=var_file['namespace'], file_folder=var_file['file_folder'])
var_source

Out[6]: '/tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab/'

In [0]:
#Origem e destino
source_lnd = "{adl_path}{lnd}/{namespace}/{file_folder}".format(adl_path=var_adls_uri, lnd=lnd, namespace=var_file["namespace"],file_folder=var_file["file_folder"])
source_lnd

Out[7]: '[REDACTED]/tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab'

In [0]:
azure_path = 'abfss://datalake@cnibigdatadlsgen2.dfs.core.windows.net'
outputh_path = '{uri}/uds/uniepro/API_InGuru/inguro_vig_acidente_trab'.format(uri=azure_path)

def API_INGURU(url, per_page, start_date, end_date):
    headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
    'accept':'application/json',
    'authorization': '6f64d6a047fad369d35c3806f8f8e0560475075a432585973f91caae35b4ad74'
    } 

    params = {
        'per_page': per_page,
        'start_date': start_date,
        'end_date': end_date,
        'sort': '1'
    }
    response = requests.get(url, headers=headers, params=params)
    return response.json()

In [0]:
def divide_data(data_inicial, data_final):
    formato_data = '%d/%m/%Y %H:%M'

    data_inicial = datetime.strptime(data_inicial, formato_data)
    data_final = datetime.strptime(data_final, formato_data)
    intervalo = data_final - data_inicial
    data_final = data_inicial + intervalo // 2
#     data_final = data_final.replace(hour=23, minute=59, second=59) # define o final como o último segundo do minuto
    data_final = data_final.strftime(formato_data)
    data_inicial = data_inicial.strftime(formato_data)
    return (data_inicial, data_final)

In [0]:
def load_big_df(path):
  schema = StructType([
    StructField("id", LongType(), nullable=True),
    StructField("domain", StringType(), nullable=True),
    StructField("title", StringType(), nullable=True),
    StructField("subtitle", StringType(), nullable=True),
    StructField("author", StringType(), nullable=True),
    StructField("content", StringType(), nullable=True),
    StructField("url", StringType(), nullable=True),
    StructField("source", StringType(), nullable=True),
    StructField("source_country", StringType(), nullable=True),
    StructField("source_state", StringType(), nullable=True),
    StructField("crawled_date", StringType(), nullable=True),
    StructField("published_date", StringType(), nullable=True),
    StructField("dh_insertion_raw", StringType(), nullable=True)
  ])

  df = spark.read.schema(schema) \
             .option("mergeSchema", "true") \
             .option("timestampFormat", "INT96") \
             .parquet(path) \
             .withColumn("dh_insertion_raw", f.col("dh_insertion_raw").cast("timestamp"))\
             .withColumn("crawled_date", f.col("crawled_date").cast("timestamp"))
  
  return df

In [0]:
def get_news(data_inicial, data_final, id):
    url = 'https://app.inguru.me/api/v1/taxonomies/nodes/news/'+str(id)

    array_dfs = []

    getted_news = 0
    
    total_news = API_INGURU(url, 1, data_inicial, data_final)['pagination']['total']
    
    if not total_news:
      dados = {
        'author': np.nan,
        'content': np.nan,
        'crawled_date': np.nan,
        'domain': np.nan,
        'id': np.nan,
        'published_date': np.nan,
        'source': np.nan,
        'source_country': np.nan,
        'source_state': np.nan,
        'subtitle': np.nan,
        'title': np.nan,
        'url': np.nan,
        'dh_insertion_raw': np.nan
      }
      fail_df = pd.DataFrame(dados, index=[0])
      array_dfs.append(fail_df)
      parcial_data_inicial = parcial_data_final
      parcial_data_final = data_final
    else:
      
      print('Total de notícias: ', total_news)

      parcial_data_inicial = data_inicial
      parcial_data_final = data_final

      while getted_news < total_news:

          parcial_news = total_news

          while parcial_news > 10000:
              print('Parcial de notícias: ', parcial_news)
              print('Data inicial: ', parcial_data_inicial)
              print('Data final: ', parcial_data_final)
              parcial_data_inicial, parcial_data_final = divide_data(parcial_data_inicial, parcial_data_final)
              parcial_news = API_INGURU(url, 1, parcial_data_inicial, parcial_data_final)['pagination']['total']

          print("fazendo requisicao")
          print('Data inicial: ', parcial_data_inicial)
          print('Data final: ', parcial_data_final)
          news = API_INGURU(url, parcial_news, parcial_data_inicial, parcial_data_final)['data']

          #CRIA UM DATAFRAME COM AS NOTICIAS
          noticias = pd.DataFrame(news)
          #ADICIONA NUM ARRAY DE DATAFRAME
          array_dfs.append(noticias)

          #corrige as datas

          parcial_data_inicial = parcial_data_final
          parcial_data_final = data_final

          getted_news += parcial_news
          

    return pd.concat(array_dfs)

In [0]:
#SALVANDO EM: /tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab_NOME_DO_NO
# /tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab_NODO_RAIZ
# /tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab_Acidentes_de_Trabalho_Judicializados
# /tmp/dev/uld/API_I

graph_ids = []
graph_names = []
graph_dfs = []

nodeID = '930' #Declaração do ID

url_nodeID = 'https://app.inguru.me/api/v1/taxonomies/nodes/'+str(nodeID)
IDs = API_INGURU(url = url_nodeID, per_page=1, start_date='', end_date='')['data'][0]
graph_ids.append(IDs['id'])
graph_names.append(IDs['name'].replace(" ", "_"))

#Passa por cada "filho" no nó principal, caso não tenha nenhum, nada acontece.
for node_children in IDs['children']:
  graph_ids.append(node_children['id'])
  graph_names.append(node_children['name'].replace(" ", "_"))


In [0]:
#Obtém as notícias de cada nó encontrado começando pelo nó principal.
#Cria um df com as notícias obtidas e salva em um array de dfs.
end_date = datetime.now().strftime("%d/%m/%Y %H:%M")
start_date = (datetime.now() - timedelta(days=2)).strftime("%d/%m/%Y %H:%M")


dh_insertion_raw = var_adf["adf_trigger_time"].split(".")[0] #Obtendo hora do sistema

In [0]:
for i, id in enumerate(graph_ids):
  url = 'https://app.inguru.me/api/v1/taxonomies/nodes/news/'+str(id)
  temp_df = get_news(start_date,end_date, id).fillna("Não informado")
  graph_df = spark.createDataFrame(temp_df)
  graph_df = graph_df.withColumn("dh_insertion_raw", f.lit(dh_insertion_raw).cast("string")) #Gravando hora em cada um dos dataframes
  if i == 0:
    path = source_lnd + "_" + str(graph_names[i]) + '_p20'
  else:
    path = source_lnd + "_" + str(graph_names[i]) + '_p0'
  
  read_path = path + str("*")
  
  old_df = load_big_df(read_path)
  
  max_date = old_df.select(f.max("crawled_date")).collect()[0][0]
  min_date = max_date - timedelta(days=3)
  print(max_date, min_date)
  old_df.filter(old_df["crawled_date"] > min_date)
  
  diff_df = graph_df.join(old_df, on='id', how='left_anti')
  
  date = datetime.today().strftime("%d%m%Y")
  write_path = path + str(f"_data={date}")
  diff_df.write.parquet(write_path, 'append')

Total de notícias: 173905
Parcial de notícias: 173905
Data inicial: 13/03/2023 04:02
Data final: 15/03/2023 04:02
Parcial de notícias: 95864
Data inicial: 13/03/2023 04:02
Data final: 14/03/2023 04:02
Parcial de notícias: 55322
Data inicial: 13/03/2023 04:02
Data final: 13/03/2023 16:02
Parcial de notícias: 18781
Data inicial: 13/03/2023 04:02
Data final: 13/03/2023 10:02
fazendo requisicao
Data inicial: 13/03/2023 04:02
Data final: 13/03/2023 07:02
Parcial de notícias: 173905
Data inicial: 13/03/2023 07:02
Data final: 15/03/2023 04:02
Parcial de notícias: 92922
Data inicial: 13/03/2023 07:02
Data final: 14/03/2023 05:32
Parcial de notícias: 64249
Data inicial: 13/03/2023 07:02
Data final: 13/03/2023 18:17
Parcial de notícias: 30429
Data inicial: 13/03/2023 07:02
Data final: 13/03/2023 12:39
Parcial de notícias: 12695
Data inicial: 13/03/2023 07:02
Data final: 13/03/2023 09:50
fazendo requisicao
Data inicial: 13/03/2023 07:02
Data final: 13/03/2023 08:26
Parcial de notícias: 173905
Data inicial: 13/03/2023 08:26
Data final: 15/03/2023 04:02
Parcial de notícias: 89386
Data inicial: 13/03/2023 08:26
Data final: 14/03/2023 06:14
Parcial de notícias: 65217
Data inicial: 13/03/2023 08:26
Data final: 13/03/2023 19:20
Parcial de notícias: 32042
Data inicial: 13/03/2023 08:26
Data final: 13/03/2023 13:53
Parcial de notícias: 16214
Data inicial: 13/03/2023 08:26
Data final: 13/03/2023 11:09
fazendo requisicao
Data inicial: 13/03/2023 08:26
Data final: 13/03/2023 09:47
Parcial de notícias: 173905
Data inicial: 13/03/2023 09:47
Data final: 15/03/2023 04:02
Parcial de notícias: 83122
Data inicial: 13/03/2023 09:47
Data final: 14/03/2023 06:54
Parcial de notícias: 62229
Data inicial: 13/03/2023 09:47
Data final: 13/03/2023 20:20
Parcial de notícias: 31875
Data inicial: 13/03/2023 09:47
Data final: 13/03/2023 15:03
Parcial de notícias: 16715
Data inicial: 13/03/2023 09:47
Data final: 13/03/2023 12:25
fazendo requisicao
Data inicial: 13/03/2023 09:47
Data final: 13/03/2023 11:06
Parcial de notícias: 173905
Data inicial: 13/03/2023 11:06
Data final: 15/03/2023 04:02
Parcial de notícias: 77049
Data inicial: 13/03/2023 11:06
Data final: 14/03/2023 07:34
Parcial de notícias: 57598
Data inicial: 13/03/2023 11:06
Data final: 13/03/2023 21:20
Parcial de notícias: 30892
Data inicial: 13/03/2023 11:06
Data final: 13/03/2023 16:13
Parcial de notícias: 14856
Data inicial: 13/03/2023 11:06
Data final: 13/03/2023 13:39
fazendo requisicao
Data inicial: 13/03/2023 11:06
Data final: 13/03/2023 12:22
Parcial de notícias: 173905
Data inicial: 13/03/2023 12:22
Data final: 15/03/2023 04:02
Parcial de notícias: 71780
Data inicial: 13/03/2023 12:22
Data final: 14/03/2023 08:12
Parcial de notícias: 52344
Data inicial: 13/03/2023 12:22
Data final: 13/03/2023 22:17
Parcial de notícias: 29666
Data inicial: 13/03/2023 12:22
Data final: 13/03/2023 17:19
Parcial de notícias: 13921
Data inicial: 13/03/2023 12:22
Data final: 13/03/2023 14:50
fazendo requisicao
Data inicial: 13/03/2023 12:22
Data final: 13/03/2023 13:36
Parcial de notícias: 173905
Data inicial: 13/03/2023 13:36
Data final: 15/03/2023 04:02
Parcial de notícias: 68195
Data inicial: 13/03/2023 13:36
Data final: 14/03/2023 08:49
Parcial de notícias: 48114
Data inicial: 13/03/2023 13:36
Data final: 13/03/2023 23:12
Parcial de notícias: 29567
Data inicial: 13/03/2023 13:36
Data final: 13/03/2023 18:24
Parcial de notícias: 14994
Data inicial: 13/03/2023 13:36
Data final: 13/03/2023 16:00
fazendo requisicao
Data inicial: 13/03/2023 13:36
Data final: 13/03/2023 14:48
Parcial de notícias: 173905
Data inicial: 13/03/2023 14:48
Data final: 15/03/2023 04:02
Parcial de notícias: 64636
Data inicial: 13/03/2023 14:48
Data final: 14/03/2023 09:25
Parcial de notícias: 42938
Data inicial: 13/03/2023 14:48
Data final: 14/03/2023 00:06
Parcial de notícias: 28222
Data inicial: 13/03/2023 14:48
Data final: 13/03/2023 19:27
Parcial de notícias: 14831
Data inicial: 13/03/2023 14:48
Data final: 13/03/2023 17:07
fazendo requisicao
D